----
Qu implementation second half
====



The steps here are somewhat straightforward:
- We take the output of a network, get the control points from it
- We generate a voronoi pattern from this, separating the image into sections where we are positive there is only one nucleus.
- We use k-means clustering for all pixels, using distance data from the nucleus and using color data to arrive at a segmentation that envelops the nucleus.

In [25]:
import cv2
import numpy as np
from tqdm import tqdm

from mask_prediction import start_over as qu
from mask_prediction import unet_semantics as model_setup
from glob import glob

def watershed(img, dist_thresh_scale=.4):
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel, iterations=1)
    sure_bg = cv2.dilate(opening, kernel, iterations=1)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, dist_thresh_scale*dist_transform.max(), 255, 0)

    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    return sure_fg, unknown

above are the imports, next come the parameters that are going to be set.

In [26]:
IMG_HEIGHT = 1024
IMG_WIDTH = 1024

nucl_rad = 90

run_name = 'diff_thresh'
dataset = 'RL012'
ini_data_path = 'X:\\BEP_data\\'                                                            #File containing data structure
em_folder = 'X:\\BEP_data\\{}\\EM'.format(dataset)                                          #Folder containing the EM datasets
ho_folder = 'X:\\BEP_data\\{}\\Hoechst'.format(dataset)                                     #Folder containing the Hoechst datasets
predict_folder = 'X:\\BEP_data\\Annotation_Iteration\\Predict_Backups\\qu_sigma2_size1e7_diameter93_80_emhof_13_2021-06-04_10-59-54\\Output'
train_folder = []
test_folder = []
mask_folder = 'X:\\BEP_data\\RL012\\Manual Masks\\'
nr_clusters = 4

data_paths = (train_folder, test_folder, em_folder, ho_folder, mask_folder)

The parameters are set, time to import the masks, and get a list of nuclei positions:

In [27]:
mask_list = glob(predict_folder + '\\*.png')
str_list = [x.split('\\')[-1] for x in mask_list]

nuclei_dict = {}

for mask in mask_list:
    img = cv2.imread(mask, cv2.IMREAD_GRAYSCALE)
    img_str = mask.split('\\')[-1]
    _, img_thresh = cv2.threshold(img, int(255*.7), 255, cv2.THRESH_BINARY)

    img_wet, unknown = watershed(img_thresh)

    # em_img = cv2.imread(em_folder + '\\Collected\\' + img_str, cv2.IMREAD_GRAYSCALE)
    # overlay_img = np.dstack((img_wet, img_thresh, em_img))

    cnts, _ = cv2.findContours(img_wet, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    ncls_pts = []
    for cnt in cnts:
        if cv2.contourArea(cnt) >= 1:
            M = cv2.moments(cnt)
            coords = [int(M['m10']/M['m00']), int(M['m01']/M['m00'])]
            ncls_pts.append(coords)
    nuclei_dict[img_str] = ncls_pts



So now we have a list for every image that we are processing of where the network thought the nuclei are. Now comes the real work, using the positions to generate a voronoi pattern.
This pattern will break up the image into segments that contain exactly one nucleus. In the next piece of code, the pattern is calculated,
and the data that will be used in k-means clustering is sandwiched. This data will consist of the EM data, the Hoechst data and a distance map based on the average diameter of the nuclei.

In [28]:
num_range = np.arange(0, 1024, 1, dtype=np.int32)
dist_limit = nucl_rad * nucl_rad
dist_limit_map = np.ones((IMG_WIDTH, IMG_HEIGHT), np.int32) * dist_limit

x_meshgrid, y_meshgrid = np.meshgrid(num_range, num_range)

for key in nuclei_dict:
    print('Currently doing {}'.format(key))

    div2d = cv2.Subdiv2D()
    div2d.initDelaunay((0,0,IMG_WIDTH, IMG_HEIGHT))
    div2d.insert(nuclei_dict[key])
    vor_list, pnts = div2d.getVoronoiFacetList([])
    dist_map_sq_tot = dist_limit_map
    label_map = np.zeros((IMG_WIDTH, IMG_HEIGHT), dtype=np.uint8)

    for pnt in nuclei_dict[key]:
        x_meshgrid_s = np.abs(x_meshgrid - pnt[0])
        y_meshgrid_s = np.abs(y_meshgrid - pnt[1])
        x_meshgrid_s = np.square(x_meshgrid_s)
        y_meshgrid_s = np.square(y_meshgrid_s)
        dist_map_sq = x_meshgrid_s + y_meshgrid_s
        dist_map_sq = np.minimum(dist_map_sq, dist_limit_map)
        dist_map_sq_tot = np.minimum(dist_map_sq, dist_map_sq_tot)

    dist_map = np.sqrt(dist_map_sq_tot)/nucl_rad
    dist_map_uint = np.array(dist_map*255, np.uint8)

    em_img = cv2.imread(em_folder + '\\Collected\\' + key, cv2.IMREAD_GRAYSCALE)
    em_bil_img = cv2.bilateralFilter(em_img, 7 , 75, 75)
    em_gauss_img = cv2.GaussianBlur(em_img, (3,3), 3)
    ho_img = cv2.imread(ho_folder + '\\Collected\\' + key, cv2.IMREAD_GRAYSCALE)
    em_show = em_img

    sandwich = np.dstack((y_meshgrid, x_meshgrid, dist_map_uint, em_gauss_img, ho_img))
    sandwich_r = np.reshape(sandwich, (-1, 5))


    for facet in tqdm(vor_list):
        facet_uint = np.array(facet, np.int32)
        mask = np.zeros((IMG_WIDTH, IMG_HEIGHT), dtype=np.uint8)
        mask = cv2.drawContours(mask, [facet_uint], -1, 255, -1, cv2.LINE_8)
        em_show = cv2.drawContours(em_show, [facet_uint], -1, 255, 2)
        mask_bool = mask == 255
        mask_bool_r = np.reshape(mask_bool, -1)

        flist = sandwich_r[mask_bool_r]
        labels = qu.color_k_means(flist, cluster_nr=nr_clusters)*(int(255/nr_clusters))
        label_map += labels

    label_floodfill = qu.get_floodfill(label_map, nuclei_dict[key])

    img_EM_clustered_floodfill = np.where(label_floodfill == 0, 255, 0)
    img_EM_clustered_floodfill = (img_EM_clustered_floodfill).astype(np.uint8)

    cv2.imwrite('X:\\BEP_data\\Annotation_Iteration\\Generated_set\\Output\\' + key, img_EM_clustered_floodfill)
model_setup.backup_data(data_paths, '*.png', run_name, 'X:\\BEP_data\\Annotation_Iteration\\Generated_set', 'X:\\BEP_data\\Annotation_Iteration\\Generated_backups', img_strs=str_list)

print('All done!')

  0%|          | 0/11 [00:00<?, ?it/s]

Currently doing 1_1_1_3.png


100%|██████████| 11/11 [00:04<00:00,  2.32it/s]


Currently doing 1_1_2_3.png


  0%|          | 0/16 [00:00<?, ?it/s]

...

Currently doing 9_4_3_3.png


  0%|          | 0/14 [00:00<?, ?it/s]

Currently doing 9_4_4_3.png


100%|██████████| 14/14 [00:04<00:00,  3.00it/s]


All done!
